In [1]:
import pandas as pd

In [13]:
data= pd.read_csv("06~23 광주아파트(매매).csv", encoding="EUC-KR")

C:\Users\gjaischool1\AppData\Local\Temp\ipykernel_14268\3144801100.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("06~23 광주아파트(매매).csv", encoding="EUC-KR")


In [4]:
data.head()

,시군구,번지,본번,부번,단지명,단지명길이,전용면적(㎡),계약년월,계약년,계약월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,광주광역시 광산구 도산동,1138-2,1138,2,대덕1,3,59.3100,202303,2023,3,18,"9,800",4,1990,송도로 143,NaN,중개거래,광주 광산구
1,광주광역시 광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202302,2023,2,7,"21,000",2,2000,송도로182번길 55,NaN,중개거래,광주 광산구
2,광주광역시 광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202305,2023,5,31,"22,350",5,2000,송도로182번길 55,NaN,중개거래,광주 광산구
3,광주광역시 광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,9,"22,400",5,2001,송도로182번길 16-3,NaN,직거래,-
4,광주광역시 광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,13,"22,000",13,2001,송도로182번길 16-3,NaN,중개거래,광주 광산구


In [17]:
data=data.drop(['번지','본번','부번','단지명길이','계약월','도로명','해제사유발생일','거래유형','중개사소재지'], axis=1)

,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,18,"9,800",4,1990
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,7,"21,000",2,2000
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,31,"22,350",5,2000
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,9,"22,400",5,2001
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,13,"22,000",13,2001


In [56]:
# 시군구에서 동명만 추출
data['동명'] = data['시군구'].apply(lambda x: x.split()[-1])

In [19]:
# 거래건수 추가
data['거래건수'] = data.groupby(['시군구', '계약년월'])['시군구'].transform('count')

In [15]:
# 건축나이 추가
data['건축나이']= data['계약년']-data['건축년도']

In [21]:
# 금리 추가
df_ir = pd.read_csv('한국은행 기준금리 및 여수신금리_2006~2023.csv')
df_ir['변환'].replace('/', '', regex=True, inplace=True)
df_ir['변환'] = df_ir['변환'].astype('int')
df_ir.set_index('변환', inplace=True)
data['기준금리'] = data['계약년월'].apply(lambda x: df_ir.loc[x, :][0])

In [22]:
data.head()

,시군구,단지명,전용면적(㎡),계약년월,계약년,계약일,거래금액(만원),층,건축년도,건축나이,동명,거래건수,기준금리
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,2023,18,"9,800",4,1990,33,도산동,20,3.5
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,2023,7,"21,000",2,2000,23,도산동,14,3.5
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,2023,31,"22,350",5,2000,23,도산동,16,3.5
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,2023,9,"22,400",5,2001,22,도산동,16,3.5
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,2023,13,"22,000",13,2001,22,도산동,16,3.5


In [27]:
# 법정명 코드 추가
bub_code=pd.read_csv('법정동코드 조회자료.csv')

In [28]:
bub_code

,법정동코드,법정동명
0,2900000000,광주광역시
1,2911000000,광주광역시 동구
2,2911010100,광주광역시 동구 대인동
3,2911010200,광주광역시 동구 금남로5가
4,2911010300,광주광역시 동구 충장로5가
...,...,...
203,2920017500,광주광역시 광산구 동호동
204,2920017600,광주광역시 광산구 덕림동
205,2920017700,광주광역시 광산구 양산동
206,2920017800,광주광역시 광산구 동림동


In [50]:
data['법정동명']=data['시군구']

In [51]:
data = data.merge(bub_code, on='법정동명', how='left')

In [37]:
data

,시군구,단지명,전용면적(㎡),계약년월,계약년,계약일,거래금액(만원),층,건축년도,건축나이,동명,거래건수,기준금리,법정동명,법정동코드_x,법정동코드_y
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,2023,18,"9,800",4,1990,33,도산동,20,3.5,광주광역시 광산구 도산동,2920010200,2920010200
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,2023,7,"21,000",2,2000,23,도산동,14,3.5,광주광역시 광산구 도산동,2920010200,2920010200
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,2023,31,"22,350",5,2000,23,도산동,16,3.5,광주광역시 광산구 도산동,2920010200,2920010200
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,2023,9,"22,400",5,2001,22,도산동,16,3.5,광주광역시 광산구 도산동,2920010200,2920010200
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,2023,13,"22,000",13,2001,22,도산동,16,3.5,광주광역시 광산구 도산동,2920010200,2920010200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425074,광주광역시 서구 화정동,화정우미,115.1850,200611,2006,10,"19,000",17,1996,10,화정동,131,4.5,광주광역시 서구 화정동,2914011900,2914011900
425075,광주광역시 서구 화정동,화정우미,84.8300,200611,2006,27,"12,000",6,1996,10,화정동,131,4.5,광주광역시 서구 화정동,2914011900,2914011900
425076,광주광역시 서구 화정동,화정우미,59.8300,200611,2006,30,"7,700",2,1996,10,화정동,131,4.5,광주광역시 서구 화정동,2914011900,2914011900
425077,광주광역시 서구 화정동,화정우미,84.8400,200612,2006,6,"12,170",16,1996,10,화정동,124,4.5,광주광역시 서구 화정동,2914011900,2914011900


In [52]:
data=data.drop(['법정동명'],axis=1)

In [54]:
data=data.drop(['건축년도','동명'],axis=1)

In [57]:
data

,시군구,단지명,전용면적(㎡),계약년월,계약년,계약일,거래금액(만원),층,건축나이,거래건수,기준금리,법정동코드,동명
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,2023,18,"9,800",4,33,20,3.5,2920010200,도산동
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,2023,7,"21,000",2,23,14,3.5,2920010200,도산동
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,2023,31,"22,350",5,23,16,3.5,2920010200,도산동
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,2023,9,"22,400",5,22,16,3.5,2920010200,도산동
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,2023,13,"22,000",13,22,16,3.5,2920010200,도산동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425074,광주광역시 서구 화정동,화정우미,115.1850,200611,2006,10,"19,000",17,10,131,4.5,2914011900,화정동
425075,광주광역시 서구 화정동,화정우미,84.8300,200611,2006,27,"12,000",6,10,131,4.5,2914011900,화정동
425076,광주광역시 서구 화정동,화정우미,59.8300,200611,2006,30,"7,700",2,10,131,4.5,2914011900,화정동
425077,광주광역시 서구 화정동,화정우미,84.8400,200612,2006,6,"12,170",16,10,124,4.5,2914011900,화정동


In [59]:
# 동별 지하철 갯수 추가
subway_count_data=pd.read_csv('동별 역갯수.csv', encoding="EUC-KR")

In [60]:
subway_count_data

,동명,지하철역수
0,광산동,1
1,금남로4가,1
2,금남로5가,1
3,농성동,2
4,도산동,1
5,마륵동,2
6,소태동,1
7,송정동,2
8,신촌동,1
9,쌍촌동,2


In [64]:
data=data.merge(subway_count_data, on="동명",how='left' )

TypeError: merge() got an unexpected keyword argument 'dtype'

In [67]:
data['지하철역수'].fillna(0, inplace=True)

In [68]:
data['지하철역수']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
425074    1.0
425075    1.0
425076    1.0
425077    1.0
425078    1.0
Name: 지하철역수, Length: 425079, dtype: float64

In [69]:
data['지하철역수'] = data['지하철역수'].astype(int)

In [73]:
column_order = ['시군구', '단지명', '전용면적(㎡)', '계약년월', '계약년', '계약일', '층', '건축나이', '거래건수', '기준금리', '법정동코드', '동명', '지하철역수', '거래금액(만원)']
data =data.reindex(columns=column_order)

In [77]:
data=data.drop(['계약년','동명'], axis=1)

In [78]:
data.to_csv('머신러닝 전처리 데이터.csv', encoding='EUC-KR')